In [103]:
from google.colab import drive, files
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re, string
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

import nltk
from nltk.corpus import stopwords


from keras.layers import Activation, Dropout, Flatten, Dense, BatchNormalization, Input
from keras.models import *
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Input
from keras.layers import Activation, Dropout, Flatten, Dense, BatchNormalization
from keras.optimizers import Adam
from tensorflow.keras.applications.resnet50 import ResNet50

In [3]:
! pip install -q kaggle

In [14]:
!mkdir covid_tweet

In [26]:
!cp kaggle.json ~/.kaggle

In [27]:
!kaggle datasets download -d datatattle/covid-19-nlp-text-classification

  0% 0.00/4.38M [00:00<?, ?B/s]
100% 4.38M/4.38M [00:00<00:00, 68.9MB/s]


In [31]:
!ls

covid-19-nlp-text-classification.zip


In [30]:
drive.mount('/content/drive/')
os.chdir('/content/drive/My Drive/Kaggle/covid_tweet')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [141]:
df = pd.read_csv("Corona_NLP_train.csv", engine='python')

In [39]:
df_train = pd.read_csv("Corona_NLP_train.csv",  engine='python')
df_train

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
0,3799,48751,London,16-03-2020,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,Neutral
1,3800,48752,UK,16-03-2020,advice Talk to your neighbours family to excha...,Positive
2,3801,48753,Vagabonds,16-03-2020,Coronavirus Australia: Woolworths to give elde...,Positive
3,3802,48754,NaN,16-03-2020,My food stock is not the only one which is emp...,Positive
4,3803,48755,NaN,16-03-2020,"Me, ready to go at supermarket during the #COV...",Extremely Negative
...,...,...,...,...,...,...
41152,44951,89903,"Wellington City, New Zealand",14-04-2020,Airline pilots offering to stock supermarket s...,Neutral
41153,44952,89904,NaN,14-04-2020,Response to complaint not provided citing COVI...,Extremely Negative
41154,44953,89905,NaN,14-04-2020,You know its getting tough when @KameronWilds...,Positive
41155,44954,89906,NaN,14-04-2020,Is it wrong that the smell of hand sanitizer i...,Neutral


In [41]:
df_train.OriginalTweet.loc[0]

'@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/iFz9FAn2Pa and https://t.co/xX6ghGFzCC and https://t.co/I2NlzdxNo8'

In [92]:
nltk.download('stopwords')

def clean_text(text):
    '''Make text lowercase then remove punctuation, username, and links'''
    text = text.lower()
    text = re.sub('@\S*|http\S*', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\w*\d\w*', '', text)
    text = re.sub('\s\s+', ' ', text)

    new_text = [x for x in text.split(' ') if x not in stopwords.words()]
    new_text = ' '.join(new_text)
    return new_text

ct = lambda x: clean_text(x)



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [93]:
stopwords

<WordListCorpusReader in '/root/nltk_data/corpora/stopwords'>

In [94]:
 text = "Airline pilots offering to stock supermarket"
 new_text = [x for x in text.split(' ') if x not in stopwords.words()]
 ' '.join(new_text)

'Airline pilots offering stock supermarket'

In [95]:
df_train.OriginalTweet = df_train.OriginalTweet.apply(ct)

In [96]:
  df_train

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
0,3799,48751,London,16-03-2020,,Neutral
1,3800,48752,UK,16-03-2020,advice talk neighbours family exchange phone n...,Positive
2,3801,48753,Vagabonds,16-03-2020,coronavirus australia woolworths give elderly ...,Positive
3,3802,48754,NaN,16-03-2020,food stock empty please dont panic enough food...,Positive
4,3803,48755,NaN,16-03-2020,ready go supermarket outbreak paranoid food st...,Extremely Negative
...,...,...,...,...,...,...
41152,44951,89903,"Wellington City, New Zealand",14-04-2020,airline pilots offering stock supermarket shel...,Neutral
41153,44952,89904,NaN,14-04-2020,response complaint provided citing related del...,Extremely Negative
41154,44953,89905,NaN,14-04-2020,know its getting tough rationing toilet paper...,Positive
41155,44954,89906,NaN,14-04-2020,wrong smell hand sanitizer starting turn coron...,Neutral


In [97]:
tokenizer = Tokenizer(oov_token='-')
tokenizer.fit_on_texts(df_train.OriginalTweet)

In [98]:
sequences = tokenizer.texts_to_sequences(df_train.OriginalTweet)
sequences[0]

[]

In [105]:
padded_sequences = pad_sequences(sequences,
                                 padding = 'post',
                                 maxlen = 20,
                                 truncating = 'post')

array([[ 0,  0,  0, ...,  0,  0,  0],
       [ 1,  1,  1, ...,  1, 12, 11],
       [ 2,  1,  1, ...,  0,  0,  0],
       ...,
       [ 1,  1,  1, ...,  0,  0,  0],
       [ 1,  1,  1, ...,  0,  0,  0],
       [ 1,  1,  1, ...,  1,  1,  1]], dtype=int32)

In [122]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(len(tokenizer.word_index), output_dim = 32),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(5, activation='softmax')
])


In [123]:
#make model folder model
model._name = "Covid_Sentiment_LSTM"
model_save_path = "./models/"
if not os.path.exists(model_save_path):
    os.mkdir(model_save_path)

In [124]:
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, Callback
checkpoint = ModelCheckpoint(os.path.join(model_save_path, model.name+".h5"), save_weights_only=True,
                             verbose=1, save_best_only=True, monitor='val_accuracy', mode='max')
early_stop = EarlyStopping(patience=7, verbose=1, monitor='val_accuracy', mode='max')

In [126]:
model.summary()

Model: "Covid_Sentiment_LSTM"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 32)          1559552   
_________________________________________________________________
lstm_1 (LSTM)                (None, 64)                24832     
_________________________________________________________________
dense_3 (Dense)              (None, 128)               8320      
_________________________________________________________________
dense_4 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_5 (Dense)              (None, 5)                 325       
Total params: 1,601,285
Trainable params: 1,601,285
Non-trainable params: 0
_________________________________________________________________


In [144]:
df_train = df_train[df_train.OriginalTweet != '']


,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
1,3800,48752,UK,16-03-2020,advice talk neighbours family exchange phone n...,Positive
2,3801,48753,Vagabonds,16-03-2020,coronavirus australia woolworths give elderly ...,Positive
3,3802,48754,NaN,16-03-2020,food stock empty please dont panic enough food...,Positive
4,3803,48755,NaN,16-03-2020,ready go supermarket outbreak paranoid food st...,Extremely Negative
5,3804,48756,"ÜT: 36.319708,-82.363649",16-03-2020,news regions first confirmed case came sulliv...,Positive
...,...,...,...,...,...,...
41152,44951,89903,"Wellington City, New Zealand",14-04-2020,airline pilots offering stock supermarket shel...,Neutral
41153,44952,89904,NaN,14-04-2020,response complaint provided citing related del...,Extremely Negative
41154,44953,89905,NaN,14-04-2020,know its getting tough rationing toilet paper...,Positive
41155,44954,89906,NaN,14-04-2020,wrong smell hand sanitizer starting turn coron...,Neutral


In [143]:
df.loc[7260].OriginalTweet

"US...Impose...New...Iran...Link...Sanctions...7...Entities...Covid-19...Consumer...Good...Giant...Face...Boycott...Call..\r\r\n.Raising...Soap...Hand...Sanitiser...Prices...Covid-19...RBI...Buy...$...1...35...bln...Worth...Gov't...Securities...Stabilise...Financial...Market..."

In [127]:
model.compile(optimizer='adam', #Using Adam Optimizer
              loss="categorical_crossentropy",   
              metrics=["accuracy"])

In [132]:
while(1):
  True

KeyboardInterrupt: ignored

In [131]:
num_epochs = 30
history = model.fit(padded_sequences, df_train["Sentiment"], epochs=num_epochs, validation_split=0.2, verbose=1)

Epoch 1/30


ValueError: ignored

In [101]:
df_train.to_csv('df_train_cleaned', index=False)

In [102]:
df_train

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
0,3799,48751,London,16-03-2020,,Neutral
1,3800,48752,UK,16-03-2020,advice talk neighbours family exchange phone n...,Positive
2,3801,48753,Vagabonds,16-03-2020,coronavirus australia woolworths give elderly ...,Positive
3,3802,48754,NaN,16-03-2020,food stock empty please dont panic enough food...,Positive
4,3803,48755,NaN,16-03-2020,ready go supermarket outbreak paranoid food st...,Extremely Negative
...,...,...,...,...,...,...
41152,44951,89903,"Wellington City, New Zealand",14-04-2020,airline pilots offering stock supermarket shel...,Neutral
41153,44952,89904,NaN,14-04-2020,response complaint provided citing related del...,Extremely Negative
41154,44953,89905,NaN,14-04-2020,know its getting tough rationing toilet paper...,Positive
41155,44954,89906,NaN,14-04-2020,wrong smell hand sanitizer starting turn coron...,Neutral
